## 📌 Passing Arguments Between Steps Using `RunnablePassthrough()`

## **🔹 What is `RunnablePassthrough()`?**
✅ **`RunnablePassthrough()`** is a special `Runnable` that simply **forwards input to the next step without modification**.  
✅ It is useful for:
- Passing **unchanged data** between steps.
- Ensuring **multiple inputs** are available later in a workflow.
- Keeping workflows **structured and readable**.


https://python.langchain.com/docs/how_to/passthrough/

---



### **👀 Example 1: Forwarding Input Without Modification**
#### **Scenario:** Keep original input while transforming text in later steps.

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableSequence


# Step 1: Passthrough (Keep Original Text)
passthrough = RunnablePassthrough()


# Step 2: Convert text to uppercase
uppercase_runnable = RunnableLambda(lambda x: {"dupliacted text": x["text"]*2})


# Step 3: Count words
word_count_runnable = RunnableLambda(lambda x: {"word_count": len(x["dupliacted text"].split())})



# Workflow: Keep original text and transform it in parallel
workflow = (
        passthrough
        | uppercase_runnable
        | word_count_runnable
            )

# Run the workflow
output = workflow.invoke({"text": "Hello LangChain!"})
print(output)

{'word_count': 3}


### **👀 Example 2: Formatting the input to the next Runnable**
#### **Scenario:** Output of the `RunnablePassthrough` is a string but the next Runnable needs a dictionary

In [25]:
import chromadb
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [26]:
# read api_key from file
with open('../api_keys.txt', 'r') as file:
    api_key = file.read()


# Set loaded api_key as OPENAI_API_KEY environmental variable
import os
os.environ["OPENAI_API_KEY"] = api_key

In [27]:
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")


llm = ChatOpenAI(model='gpt-3.5-turbo')

In [28]:
# Load the previously saved vectors
chroma_vectorstore = Chroma(embedding_function = embedding_function, 
            persist_directory='../vector_stores/FDR_State_of_Union_Embeddings')


# Create a retriever from the Chroma vector store
retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 1})  # Retrieve top match


query = "How to achieve and enjoy good health?"

retrieved_docs = retriever.invoke(query)
print(len(retrieved_docs))


# Print retrieved document contents
for doc in retrieved_docs:
    print(doc.page_content)

1
But there were no secret treaties or political or financial commitments.

The one supreme objective for the future, which we discussed for each Nation individually, and for all the United Nations, can be summed up in one word: Security.

And that means not only physical security which provides safety from attacks by aggressors. It means also economic security, social security, moral security—in a family of Nations.

In the plain down-to-earth talks that I had with the Generalissimo and Marshal Stalin and Prime Minister Churchill, it was abundantly clear that they are all most deeply interested in the resumption of peaceful progress by their own peoples—progress toward a better life. All our allies want freedom to develop their lands and resources, to build up industry, to increase education and individual opportunity, and to raise standards of living.

All our allies have learned by bitter experience that real development will not be possible if they are to be diverted from their pur

In [ ]:
template = """Answer the question based only on the following context in a few bullet points in seperate lines:
{context}

Question: {question}
"""

# The prompt expects input with keys for "context" and "question"
prompt = ChatPromptTemplate.from_template(template)


retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()} # retriever output is a string, but we need to create a dic
    | prompt
    | llm
    | StrOutputParser()
)


Question =  "How to achieve and enjoy good health?"
print(retrieval_chain.invoke(Question))

- By focusing on physical security to ensure safety from attacks
- By also focusing on economic security, social security, and moral security
- By working towards a better life through peaceful progress, development, education, and individual opportunity
- By joining together in a just and durable system of peace with all freedom-loving Nations


## 🥳 **Congratulations! You created your first RAG system!**

| **Component**           | **Role**                                           |
|-------------------------|----------------------------------------------------|
| `retriever`             | Gets relevant documents based on the question      |
| `RunnablePassthrough()` | Passes the question through unchanged              |
| `ChatPromptTemplate`    | Formats the prompt with context and question       |
| `llm`                   | Generates an answer using the formatted prompt     |
| `StrOutputParser()`     | Extracts the clean string output from the LLM      |


## **⚠️ Attention!**
```python
{"context": retriever,
 "question": RunnablePassthrough()}
```

- This is a **dictionary-style parallel runnable. (Implicit RunnableParallel via Dictionary)**
- retriever is a Runnable (e.g. `.as_retriever()`) that takes the input question and returns relevant context.
- `RunnablePassthrough()` passes the original question unchanged.